In [1]:
import pandas as pd
pd.set_option('display.max_rows', None)  # Set to None to display all rows
import urllib
import psycopg2
from psycopg2.extras import execute_values
from datetime import datetime, timedelta, timezone

from utils import get_request
from db_utils.base_utils import get_db_connection
from db_utils.prop_odds_db_utils import *

In [2]:
# Today's date (EST)
(datetime.now() - timedelta(hours=6)).strftime('%Y-%m-%d')

'2024-12-30'

In [3]:
# game_id = "c1fa7d3f30fdb408b78917509d1633c3"
# data = fetch_game_markets(game_id, 'player_shots_over_under' )
# data
# for sportsbook in data['sportsbooks']:
#     print(sportsbook)
# # Create empty dict to store outcomes by bookie
# outcomes_by_bookie = {}

# for sportsbook in data['sportsbooks']:
#     bookie_key = sportsbook['bookie_key']
#     outcomes_by_bookie[bookie_key] = []
    
#     # Get all outcomes for this sportsbook
#     for outcome in sportsbook['market']['outcomes']:
#         outcomes_by_bookie[bookie_key].append({
#             'timestamp': outcome['timestamp'],
#             'handicap': outcome['handicap'], 
#             'odds': outcome['odds'],
#             'participant_name': outcome['participant_name'],
#             'name': outcome['name'],
#             'description': outcome['description']
#         })

In [3]:
get_nhl_games_from_db('2024-12-13', enable_logging=True)

2024-12-30 10:36:03,454 - root - INFO - Retrieving NHL games from DB for date: 2024-12-13
2024-12-30 10:36:03,455 - db_utils.base_utils - INFO - Attempting connection with: host=localhost, port=5432, dbname=prop_odds_db, user=postgres
2024-12-30 10:36:03,591 - db_utils.base_utils - INFO - Successfully established database connection
2024-12-30 10:36:03,594 - root - INFO - Retrieved 1 games from the database for date 2024-12-13.


[{'id': 401688072,
  'game_id': '55280403ce6702008a627bfc4acaf1b4',
  'away_team': 'Ottawa Senators',
  'home_team': 'Carolina Hurricanes',
  'start_timestamp': datetime.datetime(2024, 12, 13, 18, 0, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=64800)))}]

In [5]:
# fetch_and_store_nhl_games('2024-12-11')

In [6]:
# get_nhl_games_from_db('2024-12-11', enable_logging=True)

In [7]:
# process_game_markets('2024-12-12', 'TOR', 'player_shots_over_under')

In [8]:
# process_nhl_games_for_date('2024-12-10')

In [9]:
# process_nhl_games_for_date('2024-12-07', enable_logging=True)

In [4]:
odds = get_player_shots_ou_odds(player_name='Artturi Lehkonen', query_date='2024-12-10', sportsbook='draftkings', team_name='Colorado Avalanche', line=True)
odds

2024-12-30 10:36:12,386 - root - INFO - Retrieving player shots OU odds for player: Artturi Lehkonen, date: 2024-12-10, sportsbook: draftkings, team: Colorado Avalanche, line: True
2024-12-30 10:36:12,387 - root - INFO - Retrieving NHL games from DB for date: 2024-12-10
2024-12-30 10:36:12,388 - db_utils.base_utils - INFO - Attempting connection with: host=localhost, port=5432, dbname=prop_odds_db, user=postgres
2024-12-30 10:36:12,415 - db_utils.base_utils - INFO - Successfully established database connection
2024-12-30 10:36:12,417 - root - INFO - Retrieved 11 games from the database for date 2024-12-10.
2024-12-30 10:36:12,418 - db_utils.base_utils - INFO - Attempting connection with: host=localhost, port=5432, dbname=prop_odds_db, user=postgres
2024-12-30 10:36:12,442 - db_utils.base_utils - INFO - Successfully established database connection
2024-12-30 10:36:12,449 - root - INFO - Completed retrieving player shots OU odds.


[{'game_id': 'ba8a3041639b65d6d085fb72d73b7b98',
  'sportsbook': 'draftkings',
  'player': 'Artturi Lehkonen',
  'ou': 'Over',
  'handicap': 2.5,
  'odds': 100,
  'timestamp': datetime.datetime(2024, 12, 11, 0, 0, 40, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=64800)))},
 {'game_id': 'ba8a3041639b65d6d085fb72d73b7b98',
  'sportsbook': 'draftkings',
  'player': 'Artturi Lehkonen',
  'ou': 'Under',
  'handicap': 2.5,
  'odds': -130,
  'timestamp': datetime.datetime(2024, 12, 11, 0, 0, 40, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=64800)))}]

In [5]:
missing = get_mismatched_game_ids_with_details(enable_logging=True)
missing

2024-12-30 10:36:14,202 - root - INFO - Comparing distinct game_ids in game_info and player_shots_ou tables.
2024-12-30 10:36:14,203 - db_utils.base_utils - INFO - Attempting connection with: host=localhost, port=5432, dbname=prop_odds_db, user=postgres
2024-12-30 10:36:14,342 - db_utils.base_utils - INFO - Successfully established database connection
2024-12-30 10:36:14,402 - root - INFO - Game IDs only in game_info: []
2024-12-30 10:36:14,403 - root - INFO - Game IDs only in player_shots_ou: set()
2024-12-30 10:36:14,403 - root - INFO - Completed comparison of game_ids.


{'only_in_game_info': [], 'only_in_player_shots_ou': []}

In [6]:
pd.DataFrame(missing['only_in_game_info'])

""


In [7]:
get_last_game_game_info(enable_logging=True)

2024-12-30 10:36:31,931 - root - INFO - Retrieving most recent game start timestamp from game_info table.
2024-12-30 10:36:31,932 - db_utils.base_utils - INFO - Attempting connection with: host=localhost, port=5432, dbname=prop_odds_db, user=postgres
2024-12-30 10:36:31,959 - db_utils.base_utils - INFO - Successfully established database connection
2024-12-30 10:36:31,961 - root - INFO - Found most recent game timestamp: 2024-12-28 21:00:00


'2024-12-28 21:00:00'

In [8]:
get_last_game_player_shots_db(enable_logging=True)

2024-12-30 10:36:39,035 - root - INFO - Retrieving most recent game info for games with player shot data.
2024-12-30 10:36:39,036 - db_utils.base_utils - INFO - Attempting connection with: host=localhost, port=5432, dbname=prop_odds_db, user=postgres
2024-12-30 10:36:39,174 - db_utils.base_utils - INFO - Successfully established database connection
2024-12-30 10:36:39,234 - root - INFO - Found most recent game info: {'game_id': '27e400124870dd763369649b68132f43', 'away_team': 'Calgary Flames', 'home_team': 'San Jose Sharks', 'start_timestamp': '2024-12-28 21:00:00'}


{'game_id': '27e400124870dd763369649b68132f43',
 'away_team': 'Calgary Flames',
 'home_team': 'San Jose Sharks',
 'start_timestamp': '2024-12-28 21:00:00'}

In [9]:
get_first_game_game_info()

2024-12-30 10:36:43,981 - db_utils.base_utils - INFO - Attempting connection with: host=localhost, port=5432, dbname=prop_odds_db, user=postgres
2024-12-30 10:36:44,116 - db_utils.base_utils - INFO - Successfully established database connection


'2024-11-01 13:00:00'

In [18]:
process_nhl_games_for_date_optimized('2024-10-22', enable_logging=False)

2024-12-30 11:27:25,496 - root - INFO - Successfully got connection from pool
2024-12-30 11:27:25,498 - db_utils.base_utils - INFO - Attempting connection with: host=localhost, port=5432, dbname=prop_odds_db, user=postgres
2024-12-30 11:27:25,525 - db_utils.base_utils - INFO - Successfully established database connection
2024-12-30 11:27:25,695 - db_utils.base_utils - INFO - Attempting connection with: host=localhost, port=5432, dbname=prop_odds_db, user=postgres
2024-12-30 11:27:25,823 - db_utils.base_utils - INFO - Successfully established database connection
2024-12-30 11:27:25,827 - db_utils.base_utils - INFO - Attempting connection with: host=localhost, port=5432, dbname=prop_odds_db, user=postgres
2024-12-30 11:27:25,962 - db_utils.base_utils - INFO - Successfully established database connection
2024-12-30 11:27:25,965 - root - INFO - Successfully got connection from pool
2024-12-30 11:27:26,103 - root - INFO - Successfully got connection from pool
